# CPython extensions for ufuncs

We proceed in steps and personal education to discern build up from first learning how to incorporate C into Python and using the Python API for 

In this notebook, we make efforts to prototype a ufunc for Python 2.7 by using CPython and the Python API. From readings, we require the following ingredient list:

### Bare minimum requirements for Python to run a C program

#### 1. Include Python API and required libraries

    #include <Python.h> 
    
    /* links the Python API as well as accompanying C libraries: <code>stdlib.h, stdio.h, errno.h</code>, do not include spaces, only included here to sidestep markdown interpreter from striking the text. */

#### 2. Function declaration

    static PyObject * mod_fun (PyObject * self, PyObject * args);

#### 3. Methods table

    static PyMethodDef ModMethods[] = {
        {"funcname", // e.g. "logit"
         mod_fun,    // links to above function declaration
         METH_VARARGS,
         "<docstring>"},
         {NULL, NULL, 0, NULL}
         }
         
    // third entry with value 0 indicates legacy PyParser is used.

#### 4. Function definition

    Static PyObject * mod_func (PyObject * self, PyObject * args)
    {
    
    // declarations for local vars needed. Suppose there are 3 arguments, then need var1, var2, var3
    
    // unpack args to local vars, for the case of 3 local vars
    if (!PyArg_ParseTuple(args, "ttt", &var1, &var2, &var3)
    {
    return NULL;
    }
    
    /* each t is a cast for each parsed argument from the tuple args. t = i, d, etc., i = integer, d = double, etc.      * The parser returns True if no errors are encountered. If there is an error in input, NULL is returned 
     * and an error is thrown. The function works in C and generates C objects.
     */
     
     // Convert C returns to PyObjects
     
     return Py_BuildValue("t", &var1); // func returns PyObject *, can we returnr PyObject ** for multiple args?
     
     // if NoneType is returned, must use PyINCREF to generate NoneType object
     

#### 5. Pass methods table to interpreter in initialization function (must be called initmod())

    PyMODINIT_FUNC
    {
        (void) Py_InitModule("mod", ModMethods);
    }
    

#### 6. Compile and link manually or create a setup.py "Makefile" to build.    

### <font color = "blue">Objective 1: create a Python-C "test" module with function $\text{expsin}(x) = e^{2x} + \sin(x)$</font>

#### testmodule.c

In [ ]:
// 1. Include Python API and required libraries
#include <Python.h>
#include <math.h>

/*
 * testmodule.c
 * This is the C code for a non-numpy Python extension to
 * define the expsin function, where expsin(p) = exp(2*x) + sin(x).
 * This function will not work on numpy arrays automatically.
 * numpy.vectorize must be called in python to generate
 * a numpy-friendly function.
 *
 * Details explaining the Python-C API can be found under
 * 'Extending and Embedding' and 'Python/C API' at
 * docs.python.org .
 */


// 2. Function declaration

static PyObject * test_expsin (PyObject * self, PyObject * args);

// 3. Methods table
static PyMethodDef TestMethods[] = {
  {"expsin",
   test_expsin,
   METH_VARARGS,
   "returns the the value expsin(p) = exp(2*x) + sin(x)"},
  {NULL, NULL, 0, NULL}

};

// 4. Function definition

  static PyObject * test_expsin (PyObject * self, PyObject * args)
  {

    double p;

    if (!PyArg_ParseTuple(args, "d", &p))
	return NULL;

    p = exp(2*p) + sin(p);

    return Py_BuildValue ("d", p);
	
  }


// 5. Initialization function 

PyMODINIT_FUNC
inittest(void)
{
  PyObject * m;
  m = Py_InitModule("test", TestMethods);
  if (m == NULL) 
    return;
}

#### setup.py ("Makefile")

In [ ]:
"""
    setup.py file for testmodule.c

    Calling
    $python setup.py build_ext --inplace
    will build the extension library in the current file.
    
    See the distutils section of
    'Extending and Embedding the Python Interpreter'
    at docs.python.org for more information.
"""


from distutils.core import setup, Extension

module1 = Extension('test', sources=['testmodule.c'],
                        include_dirs=['/usr/local/lib'])

# Extension ( 'nameofextension to be .so, if in lib/, then lib.test', sources=['filepath'])

setup(name = 'test',
        version='1.0',
        description='This is my test package',
        ext_modules = [module1])


This works for single doubles. We need to extend this to work on numpy arrays.

### <font color = "blue">Objective 2: create a ufunc for one dtype (double)</font>

#### testmodule.c

In [ ]:
// 1. Include Python API and required libraries
#include "Python.h"
#include "math.h"
#include "numpy/ndarraytypes.h"
#include "numpy/ufuncobject.h"
#include "numpy/npy_3kcompat.h"

/*
 * testmodule.c
 * This is the C code for creating your own
 * Numpy ufunc for a expsin function.
 *
 * In this code we only define the ufunc for
 * a single dtype. The computations that must
 * be replaced to create a ufunc for
 * a different function are marked with BEGIN
 * and END.
 *
 * Details explaining the Python-C API can be found under
 * 'Extending and Embedding' and 'Python/C API' at
 * docs.python.org .
 */

// 3. Methods table
static PyMethodDef TestMethods[] = {
    {NULL, NULL, 0, NULL}
};


/* The loop definition must precede the PyMODINIT_FUNC. */
static void double_expsin(char **args, npy_intp *dimensions,
                            npy_intp* steps, void* data)
{
    npy_intp i;
    npy_intp n = dimensions[0];
    printf("n = %d\n", dimensions[0]);

    char * in = args[0], char * out = args[1];
    npy_intp in_step = steps[0], out_step = steps[1];
    double tmp;

    for (i = 0; i < n; i++) {
        /*BEGIN main ufunc computation*/
        tmp = *(double *)in;
	*((double *)out)  = exp(2*tmp) + sin(tmp);
        printf("*out = %f\n", *(double *)out);
        /*END main ufunc computation*/

        in += in_step; // this is incremementing the pointer lol
        out += out_step;

        printf("*in = %f", *(double *)in);
    }
}


/*This a pointer to the above function*/
PyUFuncGenericFunction funcs[1] = {&double_expsin};

/* These are the input and return dtypes of expsin.*/
static char types[2] = {NPY_DOUBLE, NPY_DOUBLE};

/* This is extra parameters, if any, needed to compute the function eval. */
static void *data[1] = {NULL};

PyMODINIT_FUNC initnpufunc(void)
{
    PyObject *m, *expsin, *d;


    m = Py_InitModule("npufunc", TestMethods);
    if (m == NULL) {
        return;
    }

    import_array();
    import_umath();

    expsin = PyUFunc_FromFuncAndData(funcs, data, types, 1, 1, 1,
                                    PyUFunc_None, "expsin",
                                    "expsin_docstring", 0);

    d = PyModule_GetDict(m);

    PyDict_SetItemString(d, "expsin", expsin);
    Py_DECREF(expsin);
}


#### setup.py

In [ ]:
"""
    setup.py file for logit.c
    Note that since this is a numpy extension
    we use numpy.distutils instead of
    distutils from the python standard library.

    Calling
    $python setup.py build_ext --inplace
    will build the extension library in the current file.

    Calling
    $python setup.py build
    will build a file that looks like ./build/lib*, where
    lib* is a file that begins with lib. The library will
    be in this file and end with a C library extension,
    such as .so

    Calling
    $python setup.py install
    will install the module in your site-packages file.

    See the distutils section of
    'Extending and Embedding the Python Interpreter'
    at docs.python.org  and the documentation
    on numpy.distutils for more information.
"""
def configuration(parent_package='', top_path=None):
    import numpy
    from numpy.distutils.misc_util import Configuration

    config = Configuration('npufunc_directory',
                           parent_package,
                           top_path)
    config.add_extension('npufunc', ['testmodule.c'])

    return config

if __name__ == "__main__":
    from numpy.distutils.core import setup
    setup(configuration=configuration)


### <font color = "blue">Objective 3: create a ufunc for multiple dtypes (double, float, half float, ...)</font>

The construction amounts to using the same skeleton as in Objective 2, and to also include alternative versions for each dtype. No #if directives or anything needs to be defined.

#### testmodule.c

In [ ]:
// 1. Include Python API and required libraries
#include "Python.h"
#include "math.h"
#include "numpy/ndarraytypes.h"
#include "numpy/ufuncobject.h"
#include "numpy/halffloat.h"

/*
 * testmodule.c
 * This is the C code for creating your own
 * Numpy ufunc for a expsin function.
 *
 * Each function of the form type_expsin defines the
 * expsin function for a different numpy dtype. Each
 * of these functions must be modified when you
 * create your own ufunc. The computations that must
 * be replaced to create a ufunc for
 * a different funciton are marked with BEGIN
 * and END.
 *
 * Details explaining the Python-C API can be found under
 * 'Extending and Embedding' and 'Python/C API' at
 * docs.python.org .
 *
 */

// 3. Methods table
static PyMethodDef TestMethods[] = {
    {NULL, NULL, 0, NULL}
};


/* The loop definition must precede the PyMODINIT_FUNC. */
static void long_double_expsin(char **args, npy_intp *dimensions,
                            npy_intp* steps, void* data)
{
    npy_intp i;
    npy_intp n = dimensions[0];
    char * in = args[0];
    char * out = args[1];
    npy_intp in_step = steps[0]; 
    npy_intp out_step = steps[1];
    
    long double tmp;

    for (i = 0; i < n; i++) {
        /*BEGIN main ufunc computation*/
        tmp = *(long double *)in;
	*((long double *)out)  = exp(2*tmp) + sin(tmp);
        /*END main ufunc computation*/

        in += in_step; 
        out += out_step;
    }
}

static void double_expsin(char **args, npy_intp *dimensions,
                            npy_intp* steps, void* data)
{
    npy_intp i;
    npy_intp n = dimensions[0];
    char * in = args[0];
    char * out = args[1];
    npy_intp in_step = steps[0]; 
    npy_intp out_step = steps[1];
    
    double tmp;

    for (i = 0; i < n; i++) {
        /*BEGIN main ufunc computation*/
        tmp = *(double *)in;
	*((double *)out)  = exp(2*tmp) + sin(tmp);
        /*END main ufunc computation*/

        in += in_step; 
        out += out_step;
    }
}

static void float_expsin(char **args, npy_intp *dimensions,
                            npy_intp* steps, void* data)
{
    npy_intp i;
    npy_intp n = dimensions[0];
    char * in = args[0];
    char * out = args[1];
    npy_intp in_step = steps[0]; 
    npy_intp out_step = steps[1];
    
    float tmp;

    for (i = 0; i < n; i++) {
        /*BEGIN main ufunc computation*/
        tmp = *(float *)in;
	*((float *)out)  = exp(2*tmp) + sin(tmp);
        /*END main ufunc computation*/

        in += in_step; 
        out += out_step;
    }
}

static void half_float_expsin(char **args, npy_intp *dimensions,
                            npy_intp* steps, void* data)
{
    npy_intp i;
    npy_intp n = dimensions[0];
    char * in = args[0];
    char * out = args[1];
    npy_intp in_step = steps[0]; 
    npy_intp out_step = steps[1];
    
    float tmp;

    for (i = 0; i < n; i++) {
        /*BEGIN main ufunc computation*/
        tmp = *(npy_half *)in;
        tmp = npy_half_to_float(tmp);
	tmp  = exp(2*tmp) + sin(tmp);
        *((npy_half *)out) = npy_float_to_half(tmp);
        /*END main ufunc computation*/

        in += in_step; 
        out += out_step;
    }
}


/*This a pointer to the above function*/
PyUFuncGenericFunction funcs[4] = {&long_double_expsin,
				     &double_expsin,
				     &float_expsin,
				     &half_float_expsin};

/* These are the input and return dtypes of expsin.*/
static char types[8] = {NPY_LONGDOUBLE, NPY_LONGDOUBLE,
			  NPY_DOUBLE, NPY_DOUBLE,
			  NPY_FLOAT, NPY_FLOAT,
			  NPY_HALF, NPY_HALF};

/* This is extra parameters, if any, needed to compute the function eval. */
static void *data[4] = {NULL,
			  NULL,
			  NULL,
			  NULL};

PyMODINIT_FUNC initnpufunc(void)
{
    PyObject *m, *expsin, *d;


    m = Py_InitModule("npufunc", TestMethods);
    if (m == NULL) {
        return;
    }

    import_array();
    import_umath();

    expsin = PyUFunc_FromFuncAndData(funcs, data, types, 4, 1, 1,
                                    PyUFunc_None, "expsin",
                                    "expsin_docstring", 0);

// ntypes = 4 varieties of this ufunc (4 dtypes)

    d = PyModule_GetDict(m);

    PyDict_SetItemString(d, "expsin", expsin);
    Py_DECREF(expsin);
}


#### setup.py

In [ ]:
"""
    setup.py file for logit.c
    Note that since this is a numpy extension
    we use numpy.distutils instead of
    distutils from the python standard library.

    Calling
    $python setup.py build_ext --inplace
    will build the extension library in the current file.

    Calling
    $python setup.py build
    will build a file that looks like ./build/lib*, where
    lib* is a file that begins with lib. The library will
    be in this file and end with a C library extension,
    such as .so

    Calling
    $python setup.py install
    will install the module in your site-packages file.

    See the distutils section of
    'Extending and Embedding the Python Interpreter'
    at docs.python.org  and the documentation
    on numpy.distutils for more information.
"""


def configuration(parent_package='', top_path=None):
    import numpy
    from numpy.distutils.misc_util import Configuration
    from numpy.distutils.misc_util import get_info

    #Necessary for the half-float d-type.
    info = get_info('npymath')

    config = Configuration('npufunc_directory',
                            parent_package,
                            top_path)
    config.add_extension('npufunc',
                            ['testmodule.c'],
                            extra_info=info)

    return config

if __name__ == "__main__":
    from numpy.distutils.core import setup
    setup(configuration=configuration)

## <font color = "magenta">Timeit comparison</font>

Let's compare the ufunc above with the naive extension from objective 1 (i.e. we have manipulated Python to accept a C program, but it does exactly what the Python version does so there is no efficiency increase). First, note that the objective 1 fuction expsin works only on scalars. We can use a numpy.vectorize to force it to accept numpy arrays (note: this is not fancy, it <i>is</i> just (Python) looping through every single element). The function generates the trivial script needed to do this without us having to code it.

### Timeit: Objective 1 function

move into working directory

In [1]:
cd pyfiles/objective_1/

/home/dsirajud/Work/IPython-notebooks/TN-06 -- CPython extensions as ufuncs/pyfiles/objective_1


In [6]:
run setup build_ext --inplace

running build_ext
building 'test' extension
creating build
creating build/temp.linux-x86_64-2.7
gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC -I/usr/local/lib -I/usr/include/python2.7 -c testmodule.c -o build/temp.linux-x86_64-2.7/testmodule.o
gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro build/temp.linux-x86_64-2.7/testmodule.o -o /home/dsirajud/Work/IPython-notebooks/TN-06 -- CPython extensions as ufuncs/pyfiles/objective_1/test.so


Which generates the importable python extension module test.so, i.e.

In [7]:
ls

build/  setup.py  testmodule.c  test.so*


In [2]:
%%timeit
import numpy as np
import test

a = np.linspace(0,1,1000000).reshape(1000,1000)
expsin = np.vectorize(test.expsin)

expsin(a)


The slowest run took 8.06 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 155 ms per loop


### Timeit: Objective 3 function (ufunc)

move into working directory

In [3]:
cd ../objective_3/

/home/dsirajud/Work/IPython-notebooks/TN-06 -- CPython extensions as ufuncs/pyfiles/objective_3


In [11]:
run setup build_ext --inplace

running build_ext
running build_src
build_src
building extension "npufunc_directory.npufunc" sources
build_src: building npy-pkg config files
customize UnixCCompiler
customize UnixCCompiler using build_ext
building 'npufunc_directory.npufunc' extension
compiling C sources
C compiler: gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC

creating build
creating build/temp.linux-x86_64-2.7
compile options: '-I/usr/lib/python2.7/dist-packages/numpy/core/include -I/usr/lib/python2.7/dist-packages/numpy/core/include -I/usr/include/python2.7 -c'
gcc: testmodule.c
creating npufunc_directory
gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro build/temp.linux-x86_64-2.7/testmodule.o -L/usr/lib/python2.7/dist-packages/numpy/core/lib -lnpymath -lm -o npufunc_directory/npufunc.so
running scons


Which generates the importable python extension module npufunc.so contained (as we specify in setup.py in a separate directory npufunc_directory/), i.e.

In [12]:
ls

build/  npufunc_directory/  setup.py  testmodule.c


In [13]:
ls npufunc_directory/

npufunc.so*


In [4]:
cd npufunc_directory/

/home/dsirajud/Work/IPython-notebooks/TN-06 -- CPython extensions as ufuncs/pyfiles/objective_3/npufunc_directory


In [5]:
%%timeit
import numpy as np
import npufunc

a = np.linspace(0,1,1000000).reshape(1000,1000)

npufunc.expsin(a)


10 loops, best of 3: 43.7 ms per loop


## <font color = "blue">Conclusion</font>

so we see a speed increase by a factor of:

In [7]:
155 / 43.7

3.5469107551487413

which is a typical speedup that we can expect (a factor of 3 seems to be a usual floor to transferring to C from a previous Python implementation).

In this notebook, we have assembled in easy steps a python ufunc that is a precompiled C program which Python accepts and executes. In particular, the scope of ufuncs are on numpy arrays, which perform element-wise computations much faster than the interpretive execution raw Python must do.